<a href="https://colab.research.google.com/github/MoonJaeHoon/Dacon_Behavioral_Data_Analysis/blob/master/2_3%EB%B2%88_In_Ability_%EC%A0%95%EA%B7%9C%EC%8B%9D%ED%99%9C%EC%9A%A9%EC%97%86%EC%9D%B4_Generating_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
print( '변경 전 기본 경로 :' )
print(  os.getcwd() )
os.chdir('./drive/My Drive/데이콘_게임데이터분석')
print( '변경 후 파일 경로 :' )
print(  os.getcwd()  )

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook              # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
#from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings                             
warnings.filterwarnings("ignore")           # 경고 문구 미표시
%time
train = pd.read_csv('./train.csv')

#%time
#test = pd.read_pickle('./test.pkl')

변경 전 기본 경로 :
/content
변경 후 파일 경로 :
/content/drive/My Drive/데이콘_게임데이터분석
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs


## Train 으로부터 mytrain 데이터 생성

In [0]:
Ability = train.loc[train['event']=='Ability',:]
import re
def SUB(x):
    result = re.sub(' ','',str(x)) + ','
    result = re.sub('[;]','_',result)
    result = re.sub('[:]','_',result)
    #result = re.sub('[[0-9]+]',    '',     result)
    #result = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", result)
    #result = re.sub('Location:.+',    'Location:',     result)
    return result

Ability['event_contents'] = Ability.apply(lambda x: SUB(x['event_contents']), axis=1)
Ability['event_contents'].value_counts()

(1360)-TrainSCV,                                                           350376
(1820)-MorphDrone,                                                         287254
(15E0)-TrainProbe,                                                         258765
(13E0)-TrainMarine,                                                        134801
(1822)-MorphOverlord,                                                       90700
                                                                            ...  
(1AC5)-WarpInSentry_Location_(69.760986328125,113.34814453125,40912),           1
(5A0)-Attack_Location_(140.83935546875,81.169677734375,32556),                  1
(5A0)-Attack_Location_(144.671630859375,52.86572265625,40907),                  1
(1160)-ScannerSweep_Location_(122.098388671875,142.302001953125,49124),         1
(1160)-ScannerSweep_Location_(23.80859375,19.069580078125,49105),               1
Name: event_contents, Length: 906693, dtype: int64

In [0]:
game_id = train.game_id.unique().tolist()
columns = ['text0','text1','winner']
mytrain = np.zeros(len(game_id)*len(columns))
mytrain = mytrain.reshape(len(game_id),len(columns))
mytrain = pd.DataFrame(mytrain, columns=columns)
mytrain.index = game_id
mytrain.winner = train.loc[train['player']==0,:].groupby(    ['game_id'] )['winner'].max()
mytrain.loc[:,'text0'] = Ability.loc[Ability['player']==0,:].loc[Ability['event']=='Ability',:].groupby(['game_id']).event_contents.sum()
mytrain.loc[:,'text1'] = Ability.loc[Ability['player']==1,:].loc[Ability['event']=='Ability',:].groupby(['game_id']).event_contents.sum()
del train
del Ability
mytrain = mytrain.fillna('')


## Test 로부터 mytest 데이터 생성

In [0]:
test = pd.read_csv('test.csv')
Ability = test.loc[test['event']=='Ability',:]

Ability['event_contents'] = Ability.apply(lambda x: SUB(x['event_contents']), axis=1)
Ability['event_contents'].value_counts()

(1360)-TrainSCV,                                                                         153269
(1820)-MorphDrone,                                                                       122634
(15E0)-TrainProbe,                                                                       110138
(13E0)-TrainMarine,                                                                       58875
(1822)-MorphOverlord,                                                                     38779
                                                                                          ...  
(1AC1)-WarpInStalker_Location_(117.087890625,142.580322265625,49122),                         1
(5A0)-Attack_Target_Drone[04000001]_Location_(146.38037109375,22.727294921875,49120),         1
(1142)-UnloadTargetMedivac_Location_(153.00390625,15.556640625,49136),                        1
(5A0)-Attack_Location_(144.08740234375,44.3408203125,40941),                                  1
(5A0)-Attack_Location_(117.12255859375,5

In [0]:
game_id = test.game_id.unique().tolist()
columns = ['text0','text1']
mytest = np.zeros(len(game_id)*len(columns))
mytest = mytest.reshape(len(game_id),len(columns))
mytest = pd.DataFrame(mytest, columns=columns)
mytest.index = game_id

mytest.loc[:,'text0'] = Ability.loc[Ability['player']==0,:].loc[Ability['event']=='Ability',:].groupby(['game_id']).event_contents.sum()
mytest.loc[:,'text1'] = Ability.loc[Ability['player']==1,:].loc[Ability['event']=='Ability',:].groupby(['game_id']).event_contents.sum()
del test
del Ability
mytest = mytest.fillna('')


## PipeLine 없이 모델링

### 1. CountVevtorizer

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
def tokenizer(text):
    return text.split(',')

########### 사용할 모수들 ###################
min_df = 400
ngram_range = (1,1)
seed = 42

######################## 데이터로부터 단어가방 생성 ####################
vectorizer0 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews0 = [' '.join(review.split(',')) for review in mytrain.text0]
vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
print(vector_reviews0.shape)

vectorizer1 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews1 = [' '.join(review.split(',')) for review in mytrain.text1]
vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
print(vector_reviews1.shape)

player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
player0 = player0.add_prefix('player0_')
player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
player1 = player1.add_prefix('player1_')
X = pd.concat([player0,player1],axis=1)
y = mytrain.winner
print(X.shape)
del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1
del player0, player1


(38872, 750)
(38872, 757)
(38872, 1507)


In [0]:
#del X_train, X_valid, y_train, y_valid
########### 사용할 모수들 ###################
min_df = 80
seed = 42

######################## 데이터로부터 단어가방 생성 ####################
vectorizer0 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews0 = [' '.join(review.split(',')) for review in mytest.text0]
vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
print(vector_reviews0.shape)

vectorizer1 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews1 = [' '.join(review.split(',')) for review in mytest.text1]
vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
print(vector_reviews1.shape)

player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
player0 = player0.add_prefix('player0_')
player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
player1 = player1.add_prefix('player1_')

del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1

X_test = pd.concat([player0,player1],axis=1)
print(X_test.shape)

del player0, player1

(16787, 1417)
(16787, 1438)
(16787, 2855)


In [0]:
print(len(X.columns))
print(len(X_test.columns))
print(  len(    set(X.columns) & set(X_test.columns)   ) )

use_col = list(    set(X.columns) & set(X_test.columns)   )

X = X.loc[:,use_col]
X_test = X_test.loc[:,use_col]

1507
2855
1507


## 단어임베딩 저장한 pkl 불러와서 Training

In [0]:
## min_df=400

## 랜포 #  ngram (1,1) : 0.6883 
y_pred = rf.predict_proba(X_valid)[:,1]
print(rf)
print("테스트 정확도: {:.4f}".format(roc_auc_score(y_valid, y_pred)))

## LGBM #  ngram (1,1) : 0.6824
y_pred = lgbm.predict_proba(X_valid)[:,1]
print(lgbm)
print("테스트 정확도: {:.4f}".format(roc_auc_score(y_valid, y_pred)))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
테스트 정확도: 0.6839
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000,

In [0]:
X = X.add_prefix('raw_')
X_test = X_test.add_prefix('raw_')

print(X.shape)
print(X_test.shape)

X.head(2)
X_test.head(2)

,player0_40939,player1_01c80001,player0_1cc0,player0_68,player1_00240001,player0_34,player0_163,player0_1026,player0_02180001,player0_03d00001,player0_03bc0001,player0_50,player0_147,player1_1426,player1_74,player1_17a3,player1_00d40001,player0_040c0001,player0_17a3,player1_47,player1_1c80,player0_setrallypoint_location_,player1_87,player0_16e6,player0_49097,player1_03240001,player0_rootspinecrawler_location_,player0_01ac0001,player0_1261,player1_77,player0_59,player0_1542,player1_warpinstalker_location_,player0_49135,player1_researchevolvemuscularaugments,player0_upgradeterraninfantryarmor1,player0_107,player1_buildhydraliskden_location_,player0_00fc0001,player1_03b00001,...,player1_40936,player0_3480,player1_41845703125,player1_1142,player0_028c0001,player1_extrasupplies_target_supplydepot,player0_101,player1_03cc0001,player1_01980001,player1_18,player0_110,player0_upgradevehicleweapons1,player0_03a00001,player1_4160,player0_1820,player0_49122,player1_49103,player1_20a0,player0_2120,player1_00200001,player1_queentransfusion_target_queen,player1_chronoboost_target_stargate,player0_buildassimilator_target_creeponlyblocker4x4,player0_31,player1_480,player0_40931,player1_16ee,player0_01640001,player0_evolvecentrifugalhooks,player1_03a40001,player1_627685546875,player1_attack_target_overlord,player1_171,player1_setworkerrally_target_none,player1_52,player0_02dc0001,player1_1160,player0_03680001,player0_133,player1_15a0
0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [0]:
X.to_pickle('train_Ability_Raw.pkl')
X_test.to_pickle('test_Ability_Raw.pkl')